- 知識ベースを利用した ENE-NER のためのデータセットを整備
  - 整備方針は、利用を想定する素性によって決まる

# 素性 From [仲野&乾, 2018](https://www.anlp.jp/proceedings/annual_meeting/2018/pdf_dir/A5-4.pdf)

![Foo](image1.png)
- *5 部分一致から未抽出誤りまでの，固有表現として正しく抽出されなかった誤り数
- *6 過抽出も含めた全誤り数

## エラー分析

tag-wise でなく, chunk-wise に丁寧に評価
- 再現率低下(正解の固有表現を基準とした誤り; FN寄与)
  - 正解との部分一致; **知識依存**
  - クラス誤り: 範囲は完全一致する (FN as not 'O'); **文脈依存**
  - 未抽出誤り: チャンクを抽出できなかった誤り (FN as 'O'); **知識依存**
- 適合率低下(FP寄与)
  - 過抽出誤り: **文脈依存**
    - as 'O'
    - as not 'O' ⊃ サブセットでない固有表現クラスの一部に誤検出

改良方針:
- 知識依存: 全体の抽出性能を底上げするような素性の提案 → **Gazetteer 素性**
- 文脈依存: 複合語等長い固有表現や文脈の情報を上手く拾えるような素性の提案 **文節素性**


### Gazetteer 素性
- Toh ら [7] は 知識ベース内の目標単語が属しているリストの種類を素性とする Gazetteer 素性を追加
  - (他にも、外部コーパスの単語クラスタリング結果に対する、単語が所属するクラスタ番号素性を追加)
 
- Gazetteer素性の知識ベースは以下で定義:
  - 「エントリ」 := 「Wikipedia のページタイトル」( 形態素基本形 → カテゴリID if any else 0 )
  - 「エントリが属するリスト」 := 「Wikipedia のページが属するカテゴリ」  (like ENE)
    - Wikipedia 日本語版のダンプから，ページタイトルとページが属するカテゴリの ID が対応している知識べースを作成

### 文節素性
- 中野ら [9] は，目標トークンの所属文節や隣接する文節の情報を追加
  - **所属文節主辞素性**
    - 複合語は同文節になりやすい
    - 複合語の主辞は文節の主辞になりやすい
    - impl. 所属文節の主辞の基本形をクエリとして知識ベースを引き，カテゴリID(汎化素性) else 主辞基本形


- 笹野ら [10] は，
  - 目標形態素の局所的な情報（窓枠内の形態素の情報）
    - **所属文節格助詞素性**
    - impl. if 格助詞が存在しない場合 "N/A"
  - 大域的（文脈的）な情報
    - 先行文の同一形態素や共参照関係にある表現
    - 所属文節の係り先文節
      - **係り先文節主辞素性**
        - impl. 係り先文節の主辞の基本形をクエリとして知識ベースを引き，カテゴリID(汎化素性) else 主辞基本形
        - impl. if 係り先がない場合 "N/A"


## 補足. 難易度分析 From [仲野&乾, 2017](https://www.anlp.jp/proceedings/annual_meeting/2017/pdf_dir/A2-2.pdf)

### NE_Rate(x, T, D)
  - T(x) / (D(x) - T(axb))
  - コーパス Dに出現する表現 x のすべてのトークンのうち，クラスセット T に含まれるいずれかの固有表現を表しているトークンの割合
  - 値高いほど抽出が容易 (NR = 1 ならば単純な辞書ベースの手法で過不足なく抽出可能)

### Majority_Rate(x, S, T, D)
  - S_max(x) / T(x)
  - 「**人物・組織クラスサブセット**」 S (S ⊂ T) に含まれる固有表現を表しているトークンの中で，最多数を占めるクラスに属するトークンの割合
  - 値が高いほど S における固有表現クラスの曖昧性が少なく抽出が容易
  




### 難易度内訳
エラーを4象限に分類・難易度スコア（1, 2, 3）を設定
```
2(NE_Rate < Majority_Rate) 1
3 2(NE_Rate > Majority_Rate)
```

- NR>MR: コーパス中に出現すれば固有表現を表すことが多いが，固有表現クラス間の曖昧性が大きい（クラス間混同が課題）
- NR<MR: 固有表現クラス間の曖昧性は少ないが，固有表現とならないケースが多い（検出が課題）


# 素性抽出仕様

- 前処理
  - 文節まとめ上げ、主辞処理（主辞代表表記?）
  - 係り受け解析
  - 文節素性抽出器
    - **所属文節格助詞素性**

## 素性仕様
- ベース素性
  - 窓幅両側2
  - 形態素表層素性
  - 形態素品詞素性

- 知識ベース処理(「拡張固有表表現＋Wikipedia」データ)
  - **Gazetteer素性** 抽出器 (形態素単位)
    - 単一形態素 -> ENE-TYPE
    - 単一形態素 -> カテゴリID (optional)
  - 文節素性抽出器
    - **所属文節主辞素性**
    - **係り先文節主辞素性**

- 注. 文字単位素性 -> 単語単位素性にしている


# KNP( `-bnst` )ベースのチャンキング

文節内の連続する **名詞・接頭辞** に `B-NP` , `I-NP` をふる

- なんのための名詞句チャンキングか
  - キーワード抽出: 非構成的複合語の抽出
    - 教師データ必要
    - あるいは構成性を何らか定義する必要 (例. 構成性スコア `df(AB) / df(A)df(B)` )
  - 品詞区切りとしての名詞句（人間の字義通りの解釈を模倣）: 形式的定義でおおよそよい

- 考えられる形式的な名詞句の定義方法:
  - 0. 長い名詞句: 修飾要素も含めると保守的にとりすぎる
     - 「AのB」
     - 「連体修飾節A + B」
       - 「連体形複合辞（〜における、〜による）A + B」
  - 1. 文節内の連続する 名詞・接頭辞・<複合←> にふる（付属語を落とす）
     - 「AのB」「連体修飾節A + B」のような文節で区切られる名詞句が落ちる
     - 文節で区切られる「A B」名詞句のパターンを区別できる（ 業者|各数人 ）
     - 「バラク・オバマ」:（・: 付属語だが、<複合←> でとれる）

  - 2. 文節内の連続する <内容語><準内容語><複合←> にふる
     - 意味の弱い接頭辞・接尾辞が落ちる ( ご返事 -> 返事, 約半分 -> 半分, 加藤さん -> 加藤 )



- 文節の定義 := {begin: <文節始> の形態素, end: <文末>形態素 または 次の <文節始> が始まる前の形態素}
  - 基本的に <接頭> （接頭辞，括弧始） が一番強い <文節始>
  - 次いで <自立> が強い <文節始>
  - 逆に <付属> は <文節始> になり得ない
  - <複合←> も <文節始> になり得ない：<接頭>・<自立> が状況的に <文節始>になり得ないことを表す
    - Ｔ自立間連結 := 自立(→)，自立(←)にはさまれる 付属(←→)，接頭(←→)
    - (自立／Ｔ自立間連結(→)) (自立／Ｔ自立間連結(←)) の時，後者に<複合←>を与える
  - 誤って文節分かれる名詞句 AB のパターン: 名詞と名詞性形容詞語幹, その他品詞誤り
    - '西郷(名詞)隆盛(形容詞)' '新党(名詞)平和(形容詞)'
  
- 名詞
- 接頭辞
- 付属語
  - 助動詞
  - 接尾辞
  - 判定詞
  - 助詞
  - 特殊
    - (句点 読点 括弧終)
    - 各種記号
    - (＝ ・ ＆ ― ‐ … ‥ ―― …… ー 〜)
    - (↓ ↑ → ←)
    - (？ ！ ♪ ★ ☆)
    - (　)

In [22]:
from itertools import chain
from pyknp import KNP
from knp_base import KnpBase, KnpError


def __is_jiritsu(mrph):
    return ('<内容語>' in mrph.fstring or '<準内容語>' in mrph.fstring or '<複合←>' in mrph.fstring)\
        and (mrph.hinsi in {'名詞', '接尾辞', '接頭辞', '特殊'} or '<名詞的形容詞語幹>' in mrph.fstring)

def __extract_jiritsu(bnst):
    return [__is_jiritsu(m) for m in bnst.mrph_list()]

def annotate_np(blist):
    flags = chain.from_iterable([__extract_jiritsu(b) for b in blist])
    tags = []
    current_tag = 'O'
    for f in flags:
        if not f:
            tags.append('O')
            current_tag = 'O'
        elif current_tag == 'O':
            tags.append('B-NP')
            current_tag = 'B-NP'
        else:
            tags.append('I-NP')

    return tags


knp = KnpBase(knp=KNP(jumanpp=True, option='-bnst -tab'))
sentence = 'バラク・オバマ大統領がホワイトハウス入りした'
blist = knp.parse(sentence)
np_tags = annotate_np(blist)
tokens = [m.midasi for m in blist.mrph_list()]
{'tokens': tokens, 'tags': np_tags}

{'tokens': ['バラク', '・', 'オバマ', '大統領', 'が', 'ホワイトハウス', '入り', 'した'],
 'tags': ['B-NP', 'I-NP', 'I-NP', 'I-NP', 'O', 'B-NP', 'I-NP', 'O']}

以上の素性を抽出できるように、以下既存データを処理していく
- 既存データ: [BCCWJ+拡張固有表現データ](https://www.gsk.or.jp/catalog/gsk2014-a/) の形態素単位をJumanppによるものに置き換えたもの
  - 'gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type.txt'

# 「人物・組織クラスサブセット」データ From [仲野&乾, 2017](https://www.anlp.jp/proceedings/annual_meeting/2017/pdf_dir/A2-2.pdf)


第１階層（大文字）では５種類

第２階層（小文字）では 40 種類

cf.
GPE: (政府を持つ地域名), GOE: (組織名の属性を持つ施設)

- PERSON
  - Person
- ORGNIZATION
  - Organization_Other
  - International_Organization
  - Show_Organization
  - Family
  - Ethnic_Group_Other
  - Nationality
  - Sports_Organization_Other
  - Pro_Sports_Organization
  - Sports_League
  - Corporation_Other
  - Company
  - Company_Group
  - Political_Organization_Other
  - Government
  - Political_Party
  - Cabinet
  - Military
- GPE
  - GPE_Other
  - City
  - County
  - Province
  - Country
- GOE
  - GOE_Other
  - Public_Institution
  - School
  - Research_Institute
  - Market
  - Park
  - Sports_Facility
  - Museum
  - Zoo
  - Amusement_Park
  - Theater
  - Worship_Place
  - Car_Stop
  - Station
  - Airport
  - Port
- POSITION_VOCATION
  - Position_Vocation
  

In [1]:
ENE_SUBSET = {
#     'NAME':
#     {
#         'Name_Other': {},
        'Person': 'PERSON',
#         'God': {},
#         'Organization': 
#         {
            'Organization_Other': 'ORGNIZATION',
            'International_Organization': 'ORGNIZATION',
            'Show_Organization': 'ORGNIZATION',
            'Family': 'ORGNIZATION',
#             'Ethnic_Group':
#             {
                'Ethnic_Group_Other': 'ORGNIZATION',
                'Nationality': 'ORGNIZATION',
#             },
#             'Sports_Organization':
#             {
                'Sports_Organization_Other': 'ORGNIZATION',
                'Pro_Sports_Organization': 'ORGNIZATION',
                'Sports_League': 'ORGNIZATION',
#             },
#             'Corporation': 
#             {
                'Corporation_Other': 'ORGNIZATION',
                'Company': 'ORGNIZATION',
                'Company_Group': 'ORGNIZATION',
#             },
#             'Political_Organization':
#             {
                'Political_Organization_Other': 'ORGNIZATION',
                'Government': 'ORGNIZATION',
                'Political_Party': 'ORGNIZATION',
                'Cabinet': 'ORGNIZATION',
                'Military': 'ORGNIZATION',
#             }
#         },
#         'Location': 
#         {
#             'GPE':
#             {
                'GPE_Other': 'GPE',
                'City': 'GPE',
                'County': 'GPE',
                'Province': 'GPE',
                'Country': 'GPE',
#             },
#         'Facility': 
#         {
#             'GOE':
#             {
                'GOE_Other': 'GOE',
                'Public_Institution': 'GOE',
                'School': 'GOE',
                'Research_Institute': 'GOE',
                'Market': 'GOE',
                'Park': 'GOE',
                'Sports_Facility': 'GOE',
                'Museum': 'GOE',
                'Zoo': 'GOE',
                'Amusement_Park': 'GOE',
                'Theater': 'GOE',
                'Worship_Place': 'GOE',
                'Car_Stop': 'GOE',
                'Station': 'GOE',
                'Airport': 'GOE',
                'Port': 'GOE',
#             },
#         },
#         'Product': 
#         {
#             'Title':
#             {
#                 'Title_Other': {},
                'Position_Vocation': 'POSITION_VOCATION'
#             },
#         },
#     },
}

# [「拡張固有表表現＋Wikipedia」データ](http://www.languagecraft.com/enew/), [paper, 2016](https://www.anlp.jp/proceedings/annual_meeting/2016/pdf_dir/P2-4.pdf)

２種のデータが同梱
- ENE+Wデータ
- Wikipedia構造化データ

### Wikipedia構造化データ（定義）
- SID	本データにおけるID
- entry	項目名
- clean_entry	標準化された項目名 ( **KEY** )
  - 例. "entry": "EU (曖昧さ回避)", "clean_entry": "EU"
- category_info	Wikipediaのカテゴリー情報 ( **VALUE2** )
- (wikipedia_ID	WikipediaのID)
- page_property: **Normal** を絞り込むのに使う
  - 'Normal': 943512 件
  - 'Disambiguation': 「曖昧さ回避」 (40378 件)
  - 'Redirect': 「リダイレクト」(594036 件)
  - 'List': 「〜の一覧」(10464 件)

### ENE+Wデータ（定義）
- SID	本データにおけるID
- ENE	拡張固有表現 ( **VALUE1** )


In [156]:
import json


sid2dict = {}
with open('ENEW_StructuredWikipedia_20160305.txt') as f:
    for line in f:
        jd = json.loads(line)
        if jd["page_property"] == "Normal":
            sid2dict[jd['SID']] = {'entry': jd['entry'], 'clean_entry': jd['clean_entry'], 'category_info': jd['category_info']}
not_founds = []
with open('ENEW_ENEtag_20160305.txt') as f:
    for line in f:
        jd = json.loads(line)
        if jd['SID'] in sid2dict:
            sid2dict[jd['SID']].update({'ENE': jd['ENE']})
        else:
            not_founds.append(jd["SID"])
#             print(f'{jd["SID"]} not found in ENEW_Struct...')
len(sid2dict), len(not_founds)  # 1588284 -> 1079271 if 'ENE' > 0,  -> 943508 if 'Normal' -> 640506 if 'ENE' > 0

(943508, 644776)

In [157]:
len([k for k, v in sid2dict.items() if v['ENE']])

640506

In [158]:
# copied from https://sites.google.com/site/extendednamedentity711/top以下の階層の全リスト

s = """ENE | 例 | ENE英語表記
-- | -- | --
名前_その他 | たま,  ポチ,  オグリキャップ,  トントン | Name_Other
人名 | 岡本文弥,  カーン,  長門美保,  フォスター,  武帝 | Person
神名 | アテネ,  インドラ,  ゼウス,  大国主命,  帝釈天 | God
組織名(Organizaton) | 組織名_その他 | 総務課,  孔門の十哲,  向田ファミリー,  精華町町内会,  第二工学部 | Organization_Other
国際組織名 | 国際連盟,  イスラム諸国会議機構,  南太平洋フォーラム,  東南アジア条約機構 | International_Organization
公演組織名 | クリーブランド管弦楽団,  ビージーズ,  ボリショイ・バレエ団 | Show_Organization
家系名 | 久我氏,  清水家,  近衛家,  伏見宮家 | Family
民族名(Ethnic_Group) | 民族名_その他 | ケルト人,  モンゴロイド,  トラジャ（人）,  チェコ人,  アフリカーナー | Ethnic_Group_Other
国籍名 | イスラエル人,  アメリカ人,  日本国籍 | Nationality
競技組織名(Sports_Organization) | 競技組織名_その他 | 野良黒山の会,  桐山部屋,  馬家軍,  全日本,  グリーンツダ | Sports_Organization_Other
プロ競技組織名 | 読売ジャイアンツ,  ＡＣミラン,  鹿島アントラーズ,  ニューヨーク・ヤンキース | Pro_Sports_Organization
競技リーグ名 | ＮＢＡ,  セリエＡ,  セントラル・リーグ,  日本プロサッカーリーグ,  アイビー・リーグ | Sports_League
法人名(Corporation) | 法人名_その他 | 日本弁護士連合会,  宇宙開発事業団,  冷泉家時雨亭文庫,  日本相撲協会 | Corporation_Other
企業名 | ＢＭＷ,  富士電機（株）,  三菱銀行,  トステム（株）,  岩波書店（株） | Company
企業グループ名 | 三井財閥,  住友財閥,  日産コンツェルン,  ロックフェラー財閥,  デュポン財閥 | Company_Group
政治的組織名(Political_Organization) | 政治的組織名_その他 | 竹下派,  奥羽越列藩同盟,  公明市議団,  自民党府連 | Political_Organization_Other
政府組織名 | 文部省,  経済企画庁,  韓国中央情報部,  気象庁,  ナイ委員会 | Government
政党名 | ロシア共産党,  新党さきがけ,  日本新党,  共和党,  中国国民党 | Political_Party
内閣名 | 田中角栄内閣,  原敬内閣,  小渕恵三内閣,  三木武夫内閣,  西園寺公望内閣 | Cabinet
軍隊名 | 自衛隊,  アメリカ空軍,  国連軍,  多国籍軍,  連合軍 | Military
地名(Location) | 地名_その他 | タイムズ・スクエア,  グランド・ゼロ,  日本三景,  天国,  エデンの園 | Location_Other
温泉名 | 月ヶ瀬温泉,  遠刈田温泉,  白馬温泉,  福地温泉,  湯の山温泉 | Spa
ＧＰＥ(GPE) | GPE_その他 | パレスチナ自治地域,  台湾,  大阪府・市,  仏領ポリネシア | GPE_Other
市区町村名 | 月形（町）,  五箇（村）,  レッチワース,  ブランドン,  リオ・デ・ジャネイロ | City
郡名 | 真番郡,  帯方郡,  巨摩,  金堤,  固城 | County
都道府県州名 | 群馬（県）,  カンザス,  ニューヨーク（州）,  熊本（県）,  アルトア | Province
国名 | アラブ首長国連邦,  オーストラリア,  西サモア,  コートジボアール,  唐 | Country
地域名(Region) | 地域名_その他 |   | Region_Other
大陸地域名 | オリエント,  北アフリカ,  ゴンドワナ大陸,  バビロニア,  陸半球 | Continental_Region
国内地域名 | 奥羽地方,  中部地方,  カルナティック,  ボスニア,  可美 | Domestic_Region
地形名(Geological_Region) | 地形名_その他 | アルタミラ洞窟,  野島断層,  秋芳洞,  阿波の土柱,  利根川構造線 | Geological_Region_Other
山地名 | 富士山,  間ノ岳,  青崩峠,  中央アルプス,  木曽駒ケ岳 | Mountain
島名 | ラクシャドウィープ諸島,  友ヶ島,  大スンダ列島,  西表島,  沖縄諸島 | Island
河川名 | 早出川,  アーレ川,  マージー川,  千種川,  ダニューブ川 | River
湖沼名 | 大浪池,  グレート湖,  シルヤン湖,  丸沼,  サロマ湖 | Lake
海洋名 | 日本海,  バルト海,  周防灘,  関門海峡,  ホルムズ海峡 | Sea
湾名 | シェレホフ湾,  浦戸湾,  九十九湾,  ピョートル大帝湾,  ベンガル湾 | Bay
天体名(Astral_Body) | 天体名_その他 | 銀河系,  太陽系,  獅子座流星群 | Astral_Body_Other
恒星名 | アケルナル,  ウォルフ‐ライエ星,  カノープス,  ベガ,  レグルス | Star
惑星名 | 木星,  土星,  海王星,  ベスタ,  イカルス | Planet
星座名 | いて座,  エリダヌス座,  きりん座,  こと座,  ほうおう座 | Constellation
アドレス(Address) | アドレス_その他 |   | Address_Other
郵便住所 | 東京都目黒区大岡山２－１２－１,  １２３－００４５,  富士見町３－２ | Postal_Address
電話番号 | ３７２６－１１１１,  （０３）３２６９―３４７１,  内線１２３,  １１０番 | Phone_Number
電子メイル | ｓｅｋｉｎｅ＠ｃｓ．ｎｙｕ．ｅｄｕ | Email
URL | ｈｔｔｐ：／／ｃｓ．ｎｙｕ．ｅｄｕ／～ｓｅｋｉｎｅ | URL
施設名(Facility) | 施設名_その他 | 雄勝柵,  春草廬,  唐人屋敷,  三蔵,  ヘンドリク・フェアウールト・ダム | Facility_Other
施設部分名 | ８階,  南口,  １２０４号室,  華の間,  ハチ公口,  南ウィング | Facility_Part
遺跡名(Archaeological_Place) | 遺跡名_その他 | トゥルカナ遺跡,  犬伏瓦経塚,  貔子窩,  高根木戸遺跡群,  ニップール | Archaeological_Place_Other
古墳名 | 那須八幡塚古墳,  岩戸山古墳,  新山古墳,  チブサン古墳,  昭陵 | Tumulus
GOE(GOE) | GOE_その他 | ホワイトハウス,  帝国ホテル,  葵文庫,  赤坂離宮,  横田基地 | GOE_Other
公共機関名 | 黒中央郵便局,  東京家庭裁判所,  新宿駅西口交番,  高槻市役所 | Public_Institution
学校名 | プリンストン大学,  ローマ大学,  香川医科大学,  青山学院大学,  明治大学 | School
研究機関名 | ストックホルム国際平和研究所,  グリニッジ天文台,  種子島宇宙センター | Research_Institute
取引所名 | 東京証券取引所,  関西商品取引所,  神戸生糸取引所 | Market
公園名 | 上信越高原国立公園,  ザイオン国立公園,  旧円覚寺庭園,  小石川後楽園 | Park
競技施設名 | 東京ドーム,  花園ラグビー場,  石打丸山スキー場,  鳥羽CC | Sports_Facility
美術博物館名 | ルーブル美術館,  ボストン美術館,  東京国立博物館,  日本民俗資料館 | Museum
動植物園名 | 上野動物園,  ヒールズビル野生動物公園,  ニューヨーク動物公園 | Zoo
遊園施設名 | 東京ディズニーランド,  こどもの国,  チボリ公園,  ユネスコ村 | Amusement_Park
劇場名 | 明治座,  ボリショイ劇場,  パリ・オペラ座,  メトロポリタン歌劇場 | Theater
神社寺名 | 寿福寺,  サン・ドニ修道院,  円教寺,  多度神社,  スルタン・ハッサン・モスク | Worship_Place
停車場名 | 秋保神社前,  京都駅パーキングエリア,  海老名サービスエリア | Car_Stop
電車駅名 | 東京駅,  大阪駅 | Station
空港名 | 東京国際空港,  ジョン・エフ・ケネディ国際空港,  オヘア国際空港 | Airport
港名 | 神戸港,  安濃津,  十三湊,  韓泊,  横瀬浦 | Port
路線名(Line) | 路線名_その他 | 駒ケ岳ロープウェイ,  シルク・ロード | Line_Other
電車路線名 | 関西本線,  山口線,  東海道本線,  釧網本線,  宝成線 | Railroad
道路名 | 中国横断自動車道,  シルク・ロード,  ブロードウェー,  オペラ座通り,  山辺の道 | Road
運河名 | スエズ運河,  アムステルダム運河,  見沼通船堀,  セント・ローレンス水路 | Canal
航路名 | 西廻海運,  エンパイア・ルート,  青函航路,  宇高航路,  海の道 | Water_Route
トンネル名 | アペニン・トンネル,  清水トンネル,  丹那トンネル,  モファット・トンネル | Tunnel
橋名 | 瀬戸大橋,  ロンドン・ブリッジ,  万世橋,  天草五橋,  クイーンズボロ橋 | Bridge
製品名(Product) | 製品名_その他 | サランラップ,  博多人形,  縄文土器,  警察権,  金メダル,  ＧＤＰ,  米国債 | Product_Other
材料名 | ポリビニルピロリドン,  ジェット燃料,  クロムグリーン,  ナフトール染料,  オクトーゲン | Material
衣類名 | 菅笠,  木沓,  沖着物,  甲掛,  晴れ着 | Clothing
貨幣名 | ソリドゥス金貨,  デナリウス貨,  王莽銭,  アッシニャ | Money_Form
医薬品名 | 経口血糖降下薬,  アセタゾラミド,  男性ホルモン剤,  サルファ剤,  プロベネシド | Drug
武器名 | 軌道爆弾,  スプリングフィールド銃,  自走砲,  巡航ミサイル,  火炎びん | Weapon
株名 | ＮＴＴ株,  ナビスコ株,  タテホ株,  リクルートコスモス株 | Stock
賞名 | ノーベル賞,  アカデミー賞,  国民栄誉賞,  ピュリッツァー賞,  フィールズ賞 | Award
勲章名 | 文化勲章,  ガーター勲章,  レーニン勲章,  賜杯,  ブルーリボン | Decoration
罪名 | 住居侵入罪,  収賄罪,  殺人罪,  不敬罪,  マネー・ロンダリング | Offense
便名 | ノースウエスト６９便,  ひかり４０号,  インドネシア航空８７２便 | Service
等級名 | ４級,  八段,  トリプルA,  黒帯,  女子４８キロ級,  一軍 | Class
キャラクター名 | ミッキー・マウス,  ポパイ,  シンデレラ,  黄金バット | Character
識別番号 | IE１２３４－５６７８,  クモハ１２３－４５６７ | ID_Number
乗り物名(Vehicle) | 乗り物名_その他 | ラクーン,  ハーレーダビッドソン,  ウイングカスタム,  Wave125i | Vehicle_Other
車名 | カローラ,  プリマス・フューリI,  そよかぜ号 | Car
列車名 | ロコモーション号,  弁慶号,  一号機関車,  ペンドリーノ,  ＳＥ車,   零戦 | Train
飛行機名 | フライヤー号,  紫電改,  スピリット・オブ・セントルイス号,  ＦＳＸ | Aircraft
宇宙船名 | スペースシャトル,  ウォストーク,  サリュート,  スプートニク,  ミール | Spaceship
船名 | クイーン・エリザベス号,  日本丸,  ノルマンディー号,  武蔵,  高瀬舟 | Ship
食べ物名(Food) | 食べ物名_その他 | 米,  リンゴ,  水 | Food_Other
料理名 | 赤飯,  弁当,  流動食,  粥,  桶茶 | Dish
芸術作品名(Art) | 芸術作品名_その他 | ミロのヴィーナス,  聖徳太子絵伝,  聖観音菩薩像 | Art_Other
絵画名 | ゲルニカ,  モナ・リザ,  冨嶽三十六景,  イーゼンハイム祭壇画,  彦根屏風 | Picture
番組名 | 紅白歌合戦,  街頭録音 | Broadcast_Program
映画名 | 七人の侍,  モダン・タイムス,  ゴジラ男はつらいよ,  第三の男 | Movie
公演名 | おふくろ,  隅田川続俤,  通小町,  明日の幸福,  鰍沢 | Show
音楽名 | 動物の謝肉祭,  おけさ節,  魔弾の射手,  ルスランとリュドミラ,  木更津甚句 | Music
文学名 | アンタル物語,  蔭凉軒日録,  食道楽,  長ぐつ下のピッピ,  立花大全 | Book
出版物名(Printing) | 出版物名_その他 | 京都大学入学案内書,  ポケモンカレンダー,  アマルナ文書 | Printing_Other
新聞名 | 信濃毎日新聞,  ジャパンタイムズ,  官板バタヒヤ新聞,  東亜日報,  デーリー・ミラー | Newspaper
雑誌名 | 主婦の友,  四季,  ナショナル・ジオグラフィック,  馬酔木,  フォーリン・アフェアーズ | Magazine
主義方式名(Doctrine_Method) | 主義方式名_その他 | 民主主義,  護送船団方式,  育児休暇制,  アラビア文字,  ユダヤ暦,  足入れ婚 | Doctrine_Method_Other
文化名 | アシュール文化,  アンデス文明,  アメリカ美術 | Culture
宗教名 | イスラム教,  吉田神道,  天照皇大神宮教,  モルモン教,  シーア派 | Religion
学問名 | 温泉化学,  言語美学,  材料力学,  農芸化学,  量子エレクトロニクス | Academic
競技名 | 軟式テニス,  ドッジボール,  クロスカントリー・レース,  キックボクシング, かるた会 | Sport
流派名 | 一羽流,  美濃派,  文法学派,  甲源一刀流,  小原流 | Style
運動名 | ベトナム反戦運動,  沖縄復帰運動,  光州学生運動 | Movement
理論名 | 相対性理論,  進化論,  プレートテクトニクス理論 | Theory
政策計画名 | アポロ計画,  情報スーパーハイウェー構想,  ニューディール政策 | Plan
規則名(Rule) | 規則名_その他 | 公示の原則,  戦後五十年国会決議,  民法改正要綱試案 | Rule_Other
条約名 | 国際人権規約,  薩土盟約,  モスクワ協定,  パヒキナサーリ条約,  ビョルケ密約 | Treaty
法令名 | 家電リサイクル法,  御成敗式目,  労働基準法,  イスラエル破壊条項,  ２％条項 | Law
称号名(Titile) | 称号名_その他 | さん,  様,  氏,  ちゃん,  君,  殿,  夫人 | Title_Other
地位職業名 | 内閣総理大臣,  奈良奉行,  大関,  栄養士,  商人 | Position_Vocation
言語名(Language) | 言語名_その他 | 印欧語族,  ウラル・アルタイ語族,  大阪弁 | Language_Other
国語名 | 英語,  満州語,  ハンガリー語,  スウェーデン語,  チャム語 | National_Language
単位名(Unit) | 単位名_その他 | アンペア,  ニュートン,  ビット,  ロックウェル硬さ,  連 | Unit_Other
通貨単位名 | 円,  ドル,  ウォン,  フラン,  ユーロ | Currency
イベント名(Event) | イベント名_その他 | 第一次ベビーブーム,  アジア通貨危機,  北方領土問題 | Event_Other
催し物名(Occasion) | 催し物名_その他 | 筑波科学万博,  春のブライダルフェア‘９５,  ノーベル賞授賞式,  江戸の浮世絵展 | Occasion_Other
例祭名 | 端午の節供,  灯籠流し,  海神祭,  曲水宴,  チャンココ | Religious_Festival
競技会名 | 東京オリンピック,  ウィンブルドンテニス大会,  ワールドカップ,  国民体育大会 | Game
会議名 | ヨーロッパ安保協力会議,  日米構造協議,  アルカディア会談,  世界女性会議, Ｇ７ | Conference
事故事件名(Incident) | 事故事件名_その他 | 蘆溝橋事件,  ウォーターゲート事件,  生麦事件,  チャタレイ事件,  東海村臨界事故 | Incident_Other
戦争名 | 関ヶ原の戦い,  プロイセン・オーストリア戦争,  中国征伐,  宗教戦争 | War
自然現象名(Natural_Phenomenon) | 自然現象名_その他 | シベリア高気圧,  偏西風,  千島海流,  エルニーニョ | Natural_Phenomenon_Other
自然災害名 | 伊勢湾台風,  諫早豪雨,  雲仙普賢岳噴火災害,  寛永の飢饉 | Natural_Disaster
地震名 | アラスカ地震,  チリ地震,  長野県西部地震,  三河地震,  ルーマニア地震 | Earthquake
自然物名(Natural_Object) | 自然物名_その他 |   | Natural_Object_Other
元素名 | 酸素,   水素,   ヘリウム,  ルビジウム | Element
化合物名 | 塩酸,  フッ酸,  臭化アセチル,  多価アルコール,  メタロセン | Compound
鉱物名 | 苦土橄欖石,  硬マンガン鉱,  フェルグソン石,  自然銅,  芋子石 | Mineral
生物名(Living_Thing) | 生物名_その他 | インフルエンザ菌,  アンモニア化成菌,  発酵菌,  コクサッキーウイルス,  硝化細菌 | Living_Thing_Other
真菌類名 | ドクベニタケ,  ワタカビ,  バンダイキノリ,  キヌガサタケ,  ケカビ | Fungus
軟体動物_節足動物名 | コブシガニ,  サザエ,  ヤナギダコ,  アカヒトデ,  トゲサンゴ | Mollusc_Arthropod
昆虫類名 | ウラミスジシジミ,  シンクイガ,  トコジラミ,  マイマイカブリ,  ヤノネカイガラムシ | Insect
魚類名 | ニゴイ,  オヒョウ,  ホシザメ,  ヒメマス,  カワビシャ | Fish
両生類名 | サンショウウオ,  ツノガエル,  イボイモリ,  トウキョウダルマガエル,  アンヒューマ | Amphibia
爬虫類名 | ワニガメ,  アホロテトカゲ,  ヨロイハブ,  エリマキトカゲ,  クロコダイル | Reptile
鳥類名 | クジャク,  ナベヅル,  クロコシジロウミツバメ,  カンムリツクシガモ,  ウミアイサ | Bird
哺乳類名 | アメリカバイソン,  モウコウマ,  ハタリス,  オオマメジカ,  紀州犬 | Mammal
植物名 | バラ,  松,  チューリップ,  クローバー | Flora
生物部位名(Living_Thing_Part) | 生物部位名_その他 | 細胞,   染色体,   伝令RNA,   DNA | Living_Thing_Part
動物部位名 | ロレンチーニ器官,  視神経,  胎盤,  腎管,  つむじ | Animal_Part
植物部位名 | 花茎,  雌しべ,  前出葉,  機械組織,  道管 | Flora_Part
病気名(Disease) | 病気名_その他 | 黒星病,  むれ苗,  樹病 | Disease_Other
動物病気名 | ＷＰＷ症候群,  結節性多発動脈炎,  歯肉炎,  乳癌,  膀胱結核 | Animal_Disease
色名(Color) | 色名_その他 | 江戸紫,  べに色,  韓紅花,  猩々緋,  黄丹 | Color_Other
自然色名 | 赤,  青,  白,  黒,  黄 | Nature_Color
時間表現_その他 |   | Time_Top_Other
時間(Timex) | 時間_その他 | 一学期, 後期, ３時間目 | Timex_Other
時刻表現 | ３時１０分５０秒, １８：４０, 午後 １８時, 午後６時, 丑三つ時 | Time
日付表現 | ２００１年１２月１４日, ５／３１, 平成１４年, ９６年春, こどもの日 | Date
曜日表現 | 月曜, 火曜日, 水, 週末 | Day_Of_Week
時代表現 | 明治, 昭和, 江戸時代, 原始時代, ソ連時代, 若貴時代, 戦後 | Era
期間(Periodx) | 期間_その他 | ３期 | Periodx_Other
時刻期間 | ３秒間, １０分間, ５０時間, 一昼夜 | Period_Time
日数期間 | １０日間, ３０日間, 半日間, 二晩 | Period_Day
週数期間 | 一週間, ５０週間, 週 | Period_Week
月数期間 | 一ヶ月間, ４ヶ月間, １箇月間,  ひと月 | Period_Month
年数期間 | 一年間, １０年間, ５ヵ年 | Period_Year
数値表現_その他 | ８階建て, ２LDK, 二重, Ｖｅｒｓｉｏｎ６．０．５ | Numex_Other
金額表現 | １０円, 一万円, 参千円, ５０ドル | Money
株指標 | ２６　５／８ | Stock_Index
ポイント | １０点, ２２ポイント, １２得点 | Point
割合表現 | １００パーセント, 半分, 三分の二, ３割５分 | Percent
倍数表現 | ２倍, １．５倍, 百倍 | Multiplication
頻度表現 | ３回 , ８度 | Frequency
年齢 | ３ヶ月, ５歳, １８才, 還暦, ２０代後半 | Age
学齢 | １年生, 中学２年, 高三, 幼稚園児 | School_Age
序数 | 第１一次, 第１回, １７代, 第３ラウンド | Ordinal_Number
順位表現 | 一位, 優勝, ブービー, ベスト５, ８強 | Rank
緯度経度 | 北緯３０度, 西経１３５度２０分 | Latitude_Longtitude
寸法表現(Measurement) | 寸法表現_その他 | ２０ワット, ５０フォン, １０デシベル, 小さじ２, A４ | Measurement_Other
長さ | ３ミリ, ８ｍｍ, ９メートル, ５里, ８ヤード, ５尺６寸 | Physical_Extent
面積 | ２ヘクタール, ３０坪, ７平方メートル | Space
体積 | ３５０ｍｌ, １００立方メートル, 一升 | Volume
重量 | ３オンス, ５マイクロ, ３パウンド, 十貫, ５トン | Weight
速度 | 時速１００キロ, ５０ｋｍ／ｈ, ８９ヘルツ | Speed
密度 | ５キログラム毎立方メートル, ３０ｋｇ／立方メートル | Intensity
温度 | １５度, 氷点下３℃, 摂氏６度, －２℃ | Temperature
カロリー | ２０００カロリー, １２０ｋｃａｌ | Calorie
震度 | 震度４, 震度五 | Seismic_Intensity
マグニチュード | マグニチュード７．２ | Seismic_Magnitude
個数(Countx) | 個数_その他 | ２膳, ３杯, ３言語, １００万画素, １００アクセス, １０ビット | Contx_Other
人数 | ３人, 千人, ７選手, ３名人, ６閣僚 | N_Person
組織数 | １０産業グループ, １０企業, ６団体, ４組, ８家族 | N_Organization
場所数(N_Location) | 場所数_その他 | １０地域, ５領域, 二十三都道府県, １０惑星, ７ヶ所 | N_Location_Other
国数 | ２国, 四ヶ国, ３か国 | N_Country
施設数 | １０校, １０空港, ８棟, 二万戸 | N_Facility
製品数 | １０システム, ２０枚, 五万台, 三十曲, 7冊 | N_Product
イベント数 | ５つ, ４件, ３大会, ７０公演, ３勝, ４場所 | N_Event
自然物数(N_Natural_Object) | 自然物数_その他 | １０個, １０原子 | N_Natural_Object_Other
動物数 | １０匹, １０羽, １０頭 | N_Animal
植物数 | １０輪, １０本, １０株 | N_Flora
"""
en2ja_ene = dict([(l.split(' | ')[-1], l.split(' | ')[-3]) for l in s.split('\n') if ' | ' in l and len(l.split(' | '))>2][2:])
for k in ENE_SUBSET:
    assert k in en2ja_ene
ja2en_ene = {v: k for k, v in en2ja_ene.items()}


In [159]:
len(en2ja_ene), len(ja2en_ene)

(200, 200)

## 'O'以外のラベルが含まれるようにBCCWJデータをフィルタ

In [160]:


filename = 'gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type.txt'
# def dataset_reader(filename):
with open(filename) as f:
    txt = f.read()
    sentences = []
    for sentence in txt.split('\n\n'):
        sentence_ = []
        __labels = []
        for line in sentence.split('\n'):
            if len(line.split(' ')) == 4:
                surface, pos, chunk, label = line.split(' ')
                sentence_.append({'surface': surface, 'pos': pos, 'chunk': chunk, 'label': label})
                __labels.append(label)
#             else:
#                 print(line)
        if sentence_ and len(set(__labels)) > 1:
            sentences.append(sentence_)
len(sentences)  # type: 29397

29397

In [161]:
sentences_org = [sentence for sentence in sentences if any(line['label'].split('-')[1] in ENE_SUBSET for line in sentence if len(line['label'].split('-'))==2)]
len(sentences_org)  # 29497 -> 16863(人物・組織サブセット) -> 1351(Company)


16863

In [162]:
all_entities = [[line for line in sentence if len(line['label'].split('-'))==2 and line['label'].split('-')[1] in ENE_SUBSET] for sentence in sentences_org]
assert len(all_entities) == len(sentences_org)

In [163]:
# ents = [
#  [{'surface': 'バンプレスト', 'pos': '名詞', 'label': 'B-ORGANIZATION'}],
#  [{'surface': 'ヨドバシ', 'pos': '名詞', 'label': 'B-ORGANIZATION'},
#   {'surface': 'ビッグ', 'pos': '形容詞', 'label': 'B-ORGANIZATION'},
#   {'surface': 'カメラ', 'pos': '名詞', 'label': 'I-ORGANIZATION'},
#   {'surface': 'ゲオ', 'pos': '名詞', 'label': 'B-ORGANIZATION'},
#   {'surface': 'ＭＲ．', 'pos': '名詞', 'label': 'B-ORGANIZATION'},
#   {'surface': 'ＭＡＸ', 'pos': '名詞', 'label': 'I-ORGANIZATION'}],
#  [{'surface': 'ＴＯＫＵＪＩＲＯＵ', 'pos': '未定義語', 'label': 'B-ORGANIZATION'}]
# ]

def make_patterns(entities_sentences):
    patterns = []
    for entities in entities_sentences:
        surfs = []
        for ent in entities:  # ent: surface, label
            label = ent['label'].split('-')[1]
            if ent['label'].startswith('B-'):
                if surfs:
                    pattern = surfs[0] if len(surfs) == 1 else [{'lower': surf.lower()} for surf in surfs]
                    patterns.append({'label': label, 'pattern': pattern})
                surfs = [ent['surface']]
            elif ent['label'].startswith('I-'):
                surfs.append(ent['surface'])
        if surfs:
            pattern = surfs[0] if len(surfs) == 1 else [{'lower': surf.lower()} for surf in surfs]
            patterns.append({'label': label, 'pattern': pattern})

    return patterns

all_patterns = make_patterns(all_entities)

In [164]:
all_patterns[:10]

[{'label': 'Person',
  'pattern': [{'lower': '環境'},
   {'lower': '文明'},
   {'lower': '研究'},
   {'lower': '所'},
   {'lower': '所長'}]},
 {'label': 'Person', 'pattern': [{'lower': '加藤'}, {'lower': '三郎'}]},
 {'label': 'Corporation_Other',
  'pattern': [{'lower': 'ｓａｂｕｒｏ'}, {'lower': '\u3000'}, {'lower': 'ｋａｔｏ'}]},
 {'label': 'Corporation_Other', 'pattern': [{'lower': '共同'}, {'lower': '通信'}]},
 {'label': 'Position_Vocation', 'pattern': 'サラリーマン'},
 {'label': 'Position_Vocation',
  'pattern': [{'lower': 'リサイクル'}, {'lower': '事業'}, {'lower': '者'}]},
 {'label': 'Position_Vocation', 'pattern': '学者'},
 {'label': 'Position_Vocation',
  'pattern': [{'lower': 'ｏｅｃｄ'}, {'lower': '諸国'}]},
 {'label': 'Position_Vocation', 'pattern': [{'lower': '労働'}, {'lower': '者'}]},
 {'label': 'Corporation_Other',
  'pattern': [{'lower': '生活'}, {'lower': '協同'}, {'lower': '組合'}]}]

## 知識ベースをフィルタ

In [165]:
ja2en_enesub = {en2ja_ene[k]: k for k in ENE_SUBSET}
sid2dict_filtered = {}
__ignored_set = set()
for k, v in sid2dict.items():
    if v['ENE']:
        enesub = []
        for ene in v['ENE']:
            if ene in ja2en_enesub:
                enesub.append(ja2en_enesub[ene])
            elif ene not in ja2en_ene:
                __ignored_set.add(ene)
        if enesub:
            v['ENE_SUBSET'] = enesub
            sid2dict_filtered[k] = v
len(sid2dict_filtered), __ignored_set  # -> 582349, 386768(Normal)

(386768, {'CONCEPT', 'IGNORED'})

In [166]:
sid2dict_filtered[8]

{'entry': 'パリ',
 'clean_entry': 'パリ',
 'category_info': ['パリ|*', 'フランスの都市', 'ヨーロッパの首都', 'フランスの地域圏首府', '世界歴史都市連盟'],
 'ENE': ['市区町村名'],
 'ENE_SUBSET': ['City']}

## エントリを文節単位に形態素解析

### BCCWJ
- chunkingがなされていること想定

### wiki+ENE

- ID -> エントリー(複合語)

In [168]:
# from knp_base import KnpBase, KnpError

knp = KnpBase(knp=KNP(jumanpp=True, option='-bnst -tab'))

sid2entry = {sid: v['clean_entry'] for sid, v in sid2dict_filtered.items()}
sid2wakati = {}
for sid, entry in sid2entry.items():
    try:
        blist = knp.parse(entry)
        np_tags = annotate_np(blist)
        tokens = [m.midasi for m in blist.mrph_list()]
        sid2wakati[sid] = {'tokens': tokens, 'tags': np_tags}
    except KnpError:
        print(entry)
len(sid2wakati)  # 386767(Normal)

Dreamers〜EXILE VOCAL BATTLE AUDITION FINALIST〜


386767

In [169]:
# Chunkの定義 := tagが '^BI*$' となるパターン
# 誤って文節分かれる名詞句 AB のパターン: 名詞と名詞性形容詞語幹, その他品詞誤り
# '西郷(名詞)隆盛(形容詞)' '新党(名詞)平和(形容詞)'


def ordered_uniq(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if x not in seen and not seen_add(x)]

def is_chunk(tags):
    tags_uniq = ''.join(ordered_uniq(tag.split('-')[0] for tag in tags))
    return (len(tags)==1 and tags_uniq == 'B') or tags_uniq == 'BI'

sid2wakati_multi = {sid: ' '.join(d['tokens']) for sid, d in sid2wakati.items() if is_chunk(d['tags'])}
sid2wakati_rest = {sid: ' '.join(d['tokens']) for sid, d in sid2wakati.items() if not is_chunk(d['tags'])}

# sid2wakati_single = {sid: entry_w for sid, entry_w in sid2wakati.items() if len(entry_w.split(' ')) == 1}
len(sid2wakati_multi)  # 582311 -> 94507(single), 567968(multi) + 14343(rest), 380648(multi, Normal)

380648

In [170]:
list(sid2wakati_rest.items())[:30]

[(98, '村上 もと か'),
 (161, 'ここ まひ'),
 (251, 'あろ ひろし'),
 (452, '坂口 いく'),
 (1137, '本 の 雑誌 社'),
 (1313, '日本 の アナウンサー'),
 (1349, '三 山 のぼる'),
 (2381, '日本 の 空港'),
 (2432, '日本 の 行政 機関'),
 (2519, 'デニス ・ リッチー'),
 (3037, 'くぅ〜 ちゃん'),
 (5936, '第 一 勧業 銀行'),
 (7728, 'こども の 国 駅'),
 (8565, '徳川 氏'),
 (8675, '藤岡 弘 、'),
 (9760, '徳川 御 三 家'),
 (10771, '豊臣 氏'),
 (10975, '三鷹 の 森 ジブリ 美術 館'),
 (10997, '茨城 県立 取手 第 二 高等 学校'),
 (11285, 'プロフェッショナル'),
 (11921, 'ハロー ！ プロジェクト'),
 (12371, 'Ｄ － ５１'),
 (12440, '得川 氏'),
 (12523, '世良田 氏'),
 (12579, '酒井 氏'),
 (12933, '不当'),
 (12981, 'お 台場 海浜 公園 駅'),
 (13264, 'Ｑｕａｒｋ ， \u3000 Ｉｎｃ．'),
 (13417, 'アンワル ・ アッ ＝ サーダート'),
 (14099, '夢路 いとし')]

- エントリー(複合語) -> ID

In [171]:
from collections import defaultdict

entry_w2sids = defaultdict(list)
for sid, ew in sid2wakati_multi.items():
    entry_w2sids[''.join(ew.split(' '))].append(sid)
len(sid2wakati_multi), len(entry_w2sids)  # (567968, 550779) -> (380648, 368274) (Normal)

(380648, 368274)

In [172]:
entry_w2sids['東京']

[685529]

In [173]:
sid2dict_filtered[685529]

{'entry': '東京',
 'clean_entry': '東京',
 'category_info': ['日本の都市', '東京都|*', '都道府県庁所在地', '関東地方'],
 'ENE': ['市区町村名'],
 'ENE_SUBSET': ['City']}

### 名詞句候補からマッチする部分名詞句を検出

In [174]:
entry_w2sids['東京都知事']  # 東京+都+知事

[150537]

In [175]:
sid2dict_filtered[150537]

{'entry': '東京都知事',
 'clean_entry': '東京都知事',
 'category_info': ['東京都知事|*'],
 'ENE': ['地位職業名'],
 'ENE_SUBSET': ['Position_Vocation']}

In [176]:
entry_w2sids['東京都知事直轄'], entry_w2sids['地区'], entry_w2sids['邸宅']

([], [], [])

In [177]:
entry_w2sids['前東京都知事']

[]

In [47]:
def tokenize_and_chunking(entry, knp):
    blist = knp.parse(entry)
    np_tags = annotate_np(blist)
    tokens = [m.midasi for m in blist.mrph_list()]
    return tokens, np_tags

In [48]:
entry = '東京都知事直轄の地区'
tokenize_and_chunking(entry, knp)

(['東京', '都', '知事', '直轄', 'の', '地区'],
 ['B-NP', 'I-NP', 'I-NP', 'I-NP', 'O', 'B-NP'])

In [49]:
entry = '前東京都知事の邸宅'
tokenize_and_chunking(entry, knp)

(['前', '東京', '都', '知事', 'の', '邸宅'],
 ['B-NP', 'I-NP', 'I-NP', 'I-NP', 'O', 'B-NP'])

In [178]:
from knp_base import KnpBase

class KNPNPMacher:
    """ 名詞句候補集合(db)にマッチする部分名詞句を検出 """
    
    def __init__(self, db):
        self.knp = KnpBase(knp=KNP(jumanpp=True, option='-bnst -tab'))
        self.db = db

    def __tokenize_and_chunking(self, entry):
        blist = self.knp.parse(entry)
        np_tags = annotate_np(blist)
        tokens = [m.midasi for m in blist.mrph_list()]
        return tokens, np_tags

    @staticmethod
    def extract_chunks(tokens, np_tags):
        chunks = []
        chunk = {}
        current_tag = 'O'
        for idx, (token, tag) in enumerate(zip(tokens, np_tags)):
            if tag != 'O':
                if current_tag == 'O':  # B-
                    chunk['token'] = [token]
                    chunk['from'] = idx
                    current_tag = tag
                else:  # I-
                    chunk['token'].append(token)
                    current_tag = tag
            elif current_tag != 'O':  # O-
                chunk['to'] = idx
                chunks.append(chunk)
                chunk = {}
                current_tag = 'O'
        if chunk:
            chunk['to'] = idx + 1
            chunks.append(chunk)
        return chunks

    def extract_chunks_from_string(self, entry):
        tokens, np_tags = self.__tokenize_and_chunking(entry)
        return self.extract_chunks(tokens, np_tags)

    def try_matching(self, chunk):
        if ''.join(chunk) in self.db:
            return 0

        max_prefix_chunk, max_prefix_i = '', None
        max_suffix_chunk, max_suffix_i = '', None
        for i in range(1, len(chunk) + 1):
            prefix_chunk_i = ''.join(chunk[:i])
            if prefix_chunk_i in self.db:
                max_prefix_chunk = prefix_chunk_i
                max_prefix_i = i
            suffix_chunk_i = ''.join(chunk[-i:])
            if suffix_chunk_i in self.db:
                max_suffix_chunk = suffix_chunk_i
                max_suffix_i = i
        # suffix-match > prefix-match: 主辞は末尾によりがちと仮定(接尾辞が邪魔するケースは妥協)
        if max_suffix_chunk:
            return -max_suffix_i
        elif max_prefix_chunk:
            return max_prefix_i
        else:
            return None

    def match_chunks_from_string(self, entry):
        chunks = self.extract_chunks_from_string(entry)
        return self.match_chunks(chunks)
        
    def match_chunks(self, chunks):
        results = []
        for chunk in chunks:
            chunk_tokens = chunk['token']
            result_i = self.try_matching(chunk_tokens)
            if result_i is not None:
                if result_i == 0:
                    results.append( {'from': chunk['from'], 'to': chunk['to'], 'match_from': chunk['from'], 'match_to': chunk['to'], 'tokens': ' '.join(chunk_tokens), 'full_tokens': ' '.join(chunk_tokens)} )
                elif result_i < 0:
                    results.append( {'from': chunk['from'], 'to': chunk['to'], 'match_from': chunk['to'] + result_i, 'match_to': chunk['to'], 'tokens': ' '.join(chunk_tokens[result_i:]), 'full_tokens': ' '.join(chunk_tokens)} )
                else:
                    results.append( {'from': chunk['from'], 'to': chunk['to'], 'match_from': chunk['from'], 'match_to': chunk['from'] + result_i, 'tokens': ' '.join(chunk_tokens[:result_i]), 'full_tokens': ' '.join(chunk_tokens)} )
            else:
                results.append(None)
        return results



db = {'東京都知事'}
npm = KNPNPMacher(db)
entry = '東京都知事'
print(npm.match_chunks_from_string(entry))
entry = '東京都知事直轄'
print(npm.match_chunks_from_string(entry))
entry = '前東京都知事'
print(npm.match_chunks_from_string(entry))
entry = '前東京都知事直轄'
print(repr(npm.match_chunks_from_string(entry)))


[{'from': 0, 'to': 3, 'match_from': 0, 'match_to': 3, 'tokens': '東京 都 知事', 'full_tokens': '東京 都 知事'}]
[{'from': 0, 'to': 4, 'match_from': 0, 'match_to': 3, 'tokens': '東京 都 知事', 'full_tokens': '東京 都 知事 直轄'}]
[{'from': 0, 'to': 4, 'match_from': 1, 'match_to': 4, 'tokens': '東京 都 知事', 'full_tokens': '前 東京 都 知事'}]
[None]


In [179]:
db = {k for k, v in entry_w2sids.items() if v}
npm = KNPNPMacher(db)

sentence = sentences_org[6]

tokens = [line['surface'] for line in sentence]
np_tags = [line['chunk'] for line in sentence]
chunks = npm.extract_chunks(tokens, np_tags)
print(chunks)
npm.match_chunks(chunks)
# npm.match_chunks([['加藤', '三郎']])

[{'token': ['天領', '・', '日田'], 'from': 1, 'to': 4}, {'token': ['咸宜', '園'], 'from': 5, 'to': 7}, {'token': ['筑紫', '哲也'], 'from': 8, 'to': 10}, {'token': ['小鹿田焼窯主'], 'from': 12, 'to': 13}, {'token': ['坂本', '茂木'], 'from': 14, 'to': 16}]


[None,
 None,
 {'from': 8,
  'to': 10,
  'match_from': 8,
  'match_to': 10,
  'tokens': '筑紫 哲也',
  'full_tokens': '筑紫 哲也'},
 None,
 {'from': 14,
  'to': 16,
  'match_from': 14,
  'match_to': 15,
  'tokens': '坂本',
  'full_tokens': '坂本 茂木'}]

## 知識ベースエントリ素性をBCCWJデータに付与

In [180]:
from collections import Counter

db = {k for k, v in entry_w2sids.items() if v}
npm = KNPNPMacher(db)

# sid2wakati_single
# sentences_org

for sentence in sentences_org:
    for line in sentence:
        if len(line['label'].split('-')) == 2:
            enetype = line['label'].split('-')[1]
            if enetype not in ENE_SUBSET:
                line['label'] = 'O'
#         line['knowledgebase_sids'] = 'O'
        line['knowledgebase_enes'] = 'O'
    # NP-Matching
    tokens = [line['surface'] for line in sentence]
    np_tags = [line['chunk'] for line in sentence]
    chunks = npm.extract_chunks(tokens, np_tags)
    matches = npm.match_chunks(chunks)
    for m in matches:
        if m is not None:
            sids = entry_w2sids[''.join(m['tokens'].split(' '))]
            sids_str = '##'.join(map(str, sids))
#             ene_types = [[ja2en_enesub[ene] for ene in sid2dict_filtered[sid]['ENE'] if ene in ja2en_enesub] for sid in sids]
            ene_types = [ja2en_enesub[ene] for sid in sids for ene in sid2dict_filtered[sid]['ENE'] if ene in ja2en_enesub]
            # 頻出のENEタイプを選択
            ene_type = Counter(ene_types).most_common()[0][0]
                
            for idx in range(m['match_from'], m['match_to']):
                line = sentence[idx]
                if idx == m['match_from']:
#                 line['knowledgebase_sids'] = sids_str
                    line['knowledgebase_enes'] = f'B-{ene_type}'
                else:
                    line['knowledgebase_enes'] = f'I-{ene_type}'
            


In [181]:
sentences_org[6]

[{'surface': '。',
  'pos': '特殊',
  'chunk': 'O',
  'label': 'O',
  'knowledgebase_enes': 'O'},
 {'surface': '天領',
  'pos': '名詞',
  'chunk': 'B-NP',
  'label': 'B-GPE_Other',
  'knowledgebase_enes': 'O'},
 {'surface': '・',
  'pos': '特殊',
  'chunk': 'I-NP',
  'label': 'I-GPE_Other',
  'knowledgebase_enes': 'O'},
 {'surface': '日田',
  'pos': '名詞',
  'chunk': 'I-NP',
  'label': 'I-GPE_Other',
  'knowledgebase_enes': 'O'},
 {'surface': 'の',
  'pos': '助詞',
  'chunk': 'O',
  'label': 'O',
  'knowledgebase_enes': 'O'},
 {'surface': '咸宜',
  'pos': '名詞',
  'chunk': 'B-NP',
  'label': 'B-School',
  'knowledgebase_enes': 'O'},
 {'surface': '園',
  'pos': '名詞',
  'chunk': 'I-NP',
  'label': 'I-School',
  'knowledgebase_enes': 'O'},
 {'surface': 'へ',
  'pos': '助詞',
  'chunk': 'O',
  'label': 'O',
  'knowledgebase_enes': 'O'},
 {'surface': '筑紫',
  'pos': '名詞',
  'chunk': 'B-NP',
  'label': 'B-Person',
  'knowledgebase_enes': 'B-Person'},
 {'surface': '哲也',
  'pos': '名詞',
  'chunk': 'I-NP',
  'label': '

辞書が悪さする例

 {'surface': 'ランド',
  'pos': '名詞',
  'label': 'O',
  'knowledgebase_sids': '519762##1176859',
  'knowledgebase_enes': 'B-Company##B-Person'},

辞書が効く例

 {'surface': 'バンプレスト',
  'pos': '名詞',
  'label': 'B-Company',
  'knowledgebase_sids': '77140',
  'knowledgebase_enes': 'B-Company'},

In [182]:
# ファイル書き出し

#  {'surface': 'ランド',
#   'pos': '名詞',
#   'label': 'O',
#   'knowlegebase_sids': '519762##1176859',
#   'knowlegebase_enes': 'B-Company##B-Person'},

with open('gsk-ene-1.1-bccwj-json-jumanpp-type/bccwj-ene-jumanpp-type-ene-multi.txt', 'w') as f:
    for sentence in sentences_org:
        for line in sentence:
            s = f'{line["surface"]} {line["pos"]} {line["chunk"]} {line["knowledgebase_enes"]} {line["label"]}\n'
            f.write(s)
        f.write('\n')